In [2]:
import lightgbm
import pandas as pd
import numpy as np
import time
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns

import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
columns_to_normalize = ['device', 'geoNetwork','totals', 'trafficSource']
def normalize_json_data(filename):
    path = filename
    df = pd.read_csv(path, converters={column: json.loads for column in columns_to_normalize}, 
                     dtype={'fullVisitorId': 'str'})
    
    for column in columns_to_normalize:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    return df

train = normalize_json_data("train.csv")
test = normalize_json_data("test.csv")

In [5]:
train.shape


(903653, 55)

In [6]:
train.head()


,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device_browser,device_browserSize,...,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_campaignCode,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,not available in demo dataset,...,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


In [7]:
train_numerical_features = train.select_dtypes(include=[np.number])

In [8]:
test_numerical_features = test.select_dtypes(include=[np.number])

In [9]:
train_category_features = train.select_dtypes(include=[np.object])
test_category_features = test.select_dtypes(include=[np.object])
train_category_features.columns
test_category_features.columns

Index(['channelGrouping', 'fullVisitorId', 'sessionId', 'socialEngagementType',
       'device_browser', 'device_browserSize', 'device_browserVersion',
       'device_deviceCategory', 'device_flashVersion', 'device_language',
       'device_mobileDeviceBranding', 'device_mobileDeviceInfo',
       'device_mobileDeviceMarketingName', 'device_mobileDeviceModel',
       'device_mobileInputSelector', 'device_operatingSystem',
       'device_operatingSystemVersion', 'device_screenColors',
       'device_screenResolution', 'geoNetwork_city', 'geoNetwork_cityId',
       'geoNetwork_continent', 'geoNetwork_country', 'geoNetwork_latitude',
       'geoNetwork_longitude', 'geoNetwork_metro', 'geoNetwork_networkDomain',
       'geoNetwork_networkLocation', 'geoNetwork_region',
       'geoNetwork_subContinent', 'totals_bounces', 'totals_hits',
       'totals_newVisits', 'totals_pageviews', 'totals_visits',
       'trafficSource_adContent',
       'trafficSource_adwordsClickInfo.adNetworkType',
     

In [10]:
train = train.loc[:, (train != train.iloc[0]).any()]
test = test.loc[:, (test != test.iloc[0]).any()]

In [11]:
train["totals_transactionRevenue"] = train["totals_transactionRevenue"].astype('float')

In [12]:
for df in [train, test]:
    df['v_date'] = pd.to_datetime(df['visitStartTime'], unit='s')
    df['dayofweek'] = df['v_date'].dt.dayofweek
    df['hours'] = df['v_date'].dt.hour
    df['day'] = df['v_date'].dt.day
    df.drop('visitStartTime', axis=1)

In [13]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

agg_dict = {}
for col in ["totals_bounces", "totals_hits", "totals_newVisits", "totals_pageviews", "totals_transactionRevenue"]:
    train[col] = train[col].astype('float')
    agg_dict[col] = "sum"
tmp = train.groupby("fullVisitorId").agg(agg_dict).reset_index()
tmp.head()

,fullVisitorId,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_transactionRevenue
0,0000010278554503158,0.0,11.0,1.0,8.0,0.0
1,0000020424342248747,0.0,17.0,1.0,13.0,0.0
2,0000027376579751715,0.0,6.0,1.0,5.0,0.0
3,0000039460501403861,0.0,2.0,1.0,2.0,0.0
4,0000040862739425590,0.0,5.0,1.0,5.0,0.0


In [14]:
constant_columns = []
for column in train.columns:
    if len(train[column].value_counts()) == 1:
        constant_columns.append(column)

irrelevant_columns = ["visitNumber", "date", "fullVisitorId", "sessionId", "visitId", "visitStartTime", "v_date", 
                      "month", "weekday"]


In [15]:
colms = irrelevant_columns + constant_columns

In [16]:
from sklearn.preprocessing import LabelEncoder

category_columns = [c for c in train.columns if not c.startswith("total")]
category_columns = [c for c in category_columns if c not in colms]

#print(category_columns)

for c in category_columns:

    labelencode = LabelEncoder()
    train_vals = list(train[c].values.astype(str))
    test_vals = list(test[c].values.astype(str))
    
    labelencode.fit(train_vals + test_vals)
    
    train[c] = labelencode.transform(train_vals)
    test[c] = labelencode.transform(test_vals)

In [17]:
def normalize_numerical_columns(df, isTrain = True):
    df["totals_hits"] = df["totals_hits"].astype(float)
    df["totals_hits"] = (df["totals_hits"] - min(df["totals_hits"])) / (max(df["totals_hits"]) - min(df["totals_hits"]))

    df["totals_pageviews"] = df["totals_pageviews"].astype(float)
    df["totals_pageviews"] = (df["totals_pageviews"] - min(df["totals_pageviews"])) / (max(df["totals_pageviews"]) - min(df["totals_pageviews"]))
    
    if isTrain:
        df["totals_transactionRevenue"] = df["totals_transactionRevenue"].fillna(0.0)
    return df 

In [18]:
train = normalize_numerical_columns(train)
test = normalize_numerical_columns(test, isTrain = False)

In [19]:
from sklearn.model_selection import train_test_split
features = [c for c in train.columns if c not in colms]
features.remove("totals_transactionRevenue")
train["totals_transactionRevenue"] = np.log1p(train["totals_transactionRevenue"].astype(float))

In [20]:
train.head()

,channelGrouping,date,fullVisitorId,sessionId,visitId,visitNumber,visitStartTime,device_browser,device_deviceCategory,device_isMobile,...,trafficSource_campaignCode,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,v_date,dayofweek,hours,day
0,4,20160902,1131660440785968503,1131660440785968503_1472830385,1472830385,1,1472830385,35,0,0,...,NaN,NaN,11,5,3196,208,2016-09-02 15:33:05,4,7,11
1,4,20160902,377306020877927890,377306020877927890_1472880147,1472880147,1,1472880147,43,0,0,...,NaN,NaN,11,5,3196,208,2016-09-03 05:22:27,5,19,22
2,4,20160902,3895546263509774583,3895546263509774583_1472865386,1472865386,1,1472865386,35,0,0,...,NaN,NaN,11,5,3196,208,2016-09-03 01:16:26,5,1,22
3,4,20160902,4763447161404445595,4763447161404445595_1472881213,1472881213,1,1472881213,79,0,0,...,NaN,NaN,1607,5,3196,208,2016-09-03 05:40:13,5,19,22
4,4,20160902,27294437909732085,27294437909732085_1472822600,1472822600,2,1472822600,35,1,1,...,NaN,True,11,5,3196,208,2016-09-02 13:23:20,4,5,11


In [21]:
print(train.corrwith(train['totals_transactionRevenue']))

channelGrouping                                -0.000807
date                                            0.007897
visitId                                         0.010491
visitNumber                                     0.023666
visitStartTime                                  0.010491
device_browser                                 -0.046152
device_deviceCategory                          -0.042843
device_isMobile                                -0.046071
device_operatingSystem                         -0.045766
geoNetwork_city                                -0.013983
geoNetwork_continent                           -0.088376
geoNetwork_country                              0.080750
geoNetwork_metro                                0.015912
geoNetwork_networkDomain                       -0.065680
geoNetwork_region                              -0.032555
geoNetwork_subContinent                        -0.031715
totals_bounces                                       NaN
totals_hits                    

In [22]:
train_x, valid_x, train_y, valid_y = train_test_split(train[features], train["totals_transactionRevenue"], 
                                     test_size=0.25, random_state=20)

In [23]:
import lightgbm as lgb 

lgb_params = {"objective" : "regression", "metric" : "rmse",
              "num_leaves" : 50, "learning_rate" : 0.02, 
              "bagging_fraction" : 0.75, "feature_fraction" : 0.8, "bagging_frequency" : 9}
    
lgb_train = lgb.Dataset(train_x, label=train_y)
lgb_val = lgb.Dataset(valid_x, label=valid_y)
model = lgb.train(lgb_params, lgb_train, 700, valid_sets=[lgb_val], early_stopping_rounds=250, verbose_eval=100)

Training until validation scores don't improve for 250 rounds.
[100]	valid_0's rmse: 1.68274
[200]	valid_0's rmse: 1.65902
[300]	valid_0's rmse: 1.65559
[400]	valid_0's rmse: 1.65468
[500]	valid_0's rmse: 1.65422
[600]	valid_0's rmse: 1.65415
[700]	valid_0's rmse: 1.65447
Did not meet early stopping. Best iteration is:
[584]	valid_0's rmse: 1.65401


In [24]:
model = lgb.train(lgb_params, lgb_train, 700, valid_sets=[lgb_val], early_stopping_rounds=250, verbose_eval=100)

Training until validation scores don't improve for 250 rounds.
[100]	valid_0's rmse: 1.68274
[200]	valid_0's rmse: 1.65902
[300]	valid_0's rmse: 1.65559
[400]	valid_0's rmse: 1.65468
[500]	valid_0's rmse: 1.65422
[600]	valid_0's rmse: 1.65415
[700]	valid_0's rmse: 1.65447
Did not meet early stopping. Best iteration is:
[584]	valid_0's rmse: 1.65401


In [25]:
test_prediction = model.predict(test[features], num_iteration=model.best_iteration)
test["PredictedLogRevenue"] = np.expm1(test_prediction)
submission = test.groupby("fullVisitorId").agg({"PredictedLogRevenue" : "sum"}).reset_index()
submission["PredictedLogRevenue"] = np.log1p(submission["PredictedLogRevenue"])
submission["PredictedLogRevenue"] =  submission["PredictedLogRevenue"].apply(lambda x : 0.0 if x < 0 else x)
submission.to_csv("submission6.csv", index=False)
submission.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log1p
  after removing the cwd from sys.path.


,fullVisitorId,PredictedLogRevenue
0,0000000259678714014,0.127458
1,0000049363351866189,0.000000
2,0000053049821714864,0.000000
3,0000059488412965267,0.000000
4,0000085840370633780,0.017774


In [26]:
train_x, valid_x, train_y, valid_y = train_test_split(train[features], train["totals_transactionRevenue"], 
                                     test_size=0.25, random_state=20)

def permutation_test(train_X, train_y, val_X, val_y):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "num_leaves" : 50,
        "min_child_samples" : 100,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1,
        "njobs" : 3
    }
    
    lgtrain = lgb.Dataset(train_x, label=train_y)
    lgval = lgb.Dataset(valid_x, label=valid_y)
    model = lgb.train(params, lgtrain, 10000, valid_sets=[lgval], early_stopping_rounds=1000, verbose_eval=500)
    
#     pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_val_y = model.predict(valid_X, num_iteration=model.best_iteration)
    return model, pred_val_y

In [27]:
train.columns


Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime', 'device_browser',
       'device_deviceCategory', 'device_isMobile', 'device_operatingSystem',
       'geoNetwork_city', 'geoNetwork_continent', 'geoNetwork_country',
       'geoNetwork_metro', 'geoNetwork_networkDomain', 'geoNetwork_region',
       'geoNetwork_subContinent', 'totals_bounces', 'totals_hits',
       'totals_newVisits', 'totals_pageviews', 'totals_transactionRevenue',
       'trafficSource_adContent',
       'trafficSource_adwordsClickInfo.adNetworkType',
       'trafficSource_adwordsClickInfo.gclId',
       'trafficSource_adwordsClickInfo.isVideoAd',
       'trafficSource_adwordsClickInfo.page',
       'trafficSource_adwordsClickInfo.slot', 'trafficSource_campaign',
       'trafficSource_campaignCode', 'trafficSource_isTrueDirect',
       'trafficSource_keyword', 'trafficSource_medium',
       'trafficSource_referralPath', 'trafficSource_source', 'v_date',


In [38]:
from datetime import *
List = ["totals_pageviews", "totals_hits", "visitNumber", "geoNetwork_country"]
for column_perm in List:
    for i in range(20):
        print(column_perm," ",i)
        train_permute = train.copy()
        train_permute[column_perm] = np.random.permutation(train_permute[column_perm])
        dev_df = train_permute[train_permute['date']<=20170531]
        val_df = train_permute[train_permute['date']>20170531]
        dev_y = np.log1p(dev_df["totals_transactionRevenue"].values)
        val_y = np.log1p(val_df["totals_transactionRevenue"].values)

        dev_X = dev_df[cat_cols + num_cols] 
        val_X = val_df[cat_cols + num_cols] 
        model, pred_val = permutation_test(dev_X, dev_y, val_X, val_y)

totals_pageviews   0


NameError: name 'cat_cols' is not defined

In [42]:
train.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId', 'visitId',
       'visitNumber', 'visitStartTime', 'device_browser',
       'device_deviceCategory', 'device_isMobile', 'device_operatingSystem',
       'geoNetwork_city', 'geoNetwork_continent', 'geoNetwork_country',
       'geoNetwork_metro', 'geoNetwork_networkDomain', 'geoNetwork_region',
       'geoNetwork_subContinent', 'totals_bounces', 'totals_hits',
       'totals_newVisits', 'totals_pageviews', 'totals_transactionRevenue',
       'trafficSource_adContent',
       'trafficSource_adwordsClickInfo.adNetworkType',
       'trafficSource_adwordsClickInfo.gclId',
       'trafficSource_adwordsClickInfo.isVideoAd',
       'trafficSource_adwordsClickInfo.page',
       'trafficSource_adwordsClickInfo.slot', 'trafficSource_campaign',
       'trafficSource_campaignCode', 'trafficSource_isTrueDirect',
       'trafficSource_keyword', 'trafficSource_medium',
       'trafficSource_referralPath', 'trafficSource_source', 'v_date',


In [43]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 40 columns):
channelGrouping                                 903653 non-null int64
date                                            903653 non-null int64
fullVisitorId                                   903653 non-null object
sessionId                                       903653 non-null object
visitId                                         903653 non-null int64
visitNumber                                     903653 non-null int64
visitStartTime                                  903653 non-null int64
device_browser                                  903653 non-null int64
device_deviceCategory                           903653 non-null int64
device_isMobile                                 903653 non-null int64
device_operatingSystem                          903653 non-null int64
geoNetwork_city                                 903653 non-null int64
geoNetwork_continent                           

In [45]:
numerical_colums = ["totals_hits", "totals_bounces", "totals_pageviews", "totals_newVisits", "visitNumber"]

In [46]:
category_columns

['channelGrouping',
 'device_browser',
 'device_deviceCategory',
 'device_isMobile',
 'device_operatingSystem',
 'geoNetwork_city',
 'geoNetwork_continent',
 'geoNetwork_country',
 'geoNetwork_metro',
 'geoNetwork_networkDomain',
 'geoNetwork_region',
 'geoNetwork_subContinent',
 'trafficSource_adContent',
 'trafficSource_adwordsClickInfo.adNetworkType',
 'trafficSource_adwordsClickInfo.gclId',
 'trafficSource_adwordsClickInfo.page',
 'trafficSource_adwordsClickInfo.slot',
 'trafficSource_campaign',
 'trafficSource_keyword',
 'trafficSource_medium',
 'trafficSource_referralPath',
 'trafficSource_source',
 'dayofweek',
 'hours',
 'day']

In [47]:
from datetime import *
List = ["totals_pageviews", "totals_hits", "visitNumber", "geoNetwork_country"]
for column_perm in List:
    for i in range(20):
        print(column_perm," ",i)
        train_permute = train.copy()
        train_permute[column_perm] = np.random.permutation(train_permute[column_perm])
        dev_df = train_permute[train_permute['date']<=20170531]
        val_df = train_permute[train_permute['date']>20170531]
        dev_y = np.log1p(dev_df["totals_transactionRevenue"].values)
        val_y = np.log1p(val_df["totals_transactionRevenue"].values)

        dev_X = dev_df[category_columns + numerical_colums] 
        val_X = val_df[category_columns + numerical_colums] 
        model, pred_val = permutation_test(dev_X, dev_y, val_X, val_y)

totals_pageviews   0


LightGBMError: Length of label is not same with #data

In [50]:
model = lgb.train(params, lgtrain, 10000, valid_sets=[lgval], early_stopping_rounds=1000, verbose_eval=500)

NameError: name 'lgtrain' is not defined

In [49]:
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "num_leaves" : 50,
        "min_child_samples" : 100,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1,
        "njobs" : 3
    }

In [51]:
lgb_params = {"objective" : "regression", "metric" : "rmse",
              "num_leaves" : 50, "learning_rate" : 0.02, 
              "bagging_fraction" : 0.75, "feature_fraction" : 0.8, "bagging_frequency" : 9}
    
lgb_train = lgb.Dataset(train_x, label=train_y)
lgb_val = lgb.Dataset(valid_x, label=valid_y)
model = lgb.train(lgb_params, lgb_train, 700, valid_sets=[lgb_val], early_stopping_rounds=250, verbose_eval=100)

Training until validation scores don't improve for 250 rounds.
[100]	valid_0's rmse: 1.68274
[200]	valid_0's rmse: 1.65902
[300]	valid_0's rmse: 1.65559
[400]	valid_0's rmse: 1.65468
[500]	valid_0's rmse: 1.65422
[600]	valid_0's rmse: 1.65415
[700]	valid_0's rmse: 1.65447
Did not meet early stopping. Best iteration is:
[584]	valid_0's rmse: 1.65401


In [56]:
train_x, valid_x, train_y, valid_y = train_test_split(train[features], train["totals_transactionRevenue"], 
                                     test_size=0.25, random_state=20)

def permutation_test(train_X, train_y, val_X, val_y):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "num_leaves" : 50,
        "min_child_samples" : 100,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1,
        "njobs" : 3
    }
    
    lgtrain = lgb.Dataset(train_x, label=train_y)
    lgval = lgb.Dataset(valid_x, label=valid_y)
    model = lgb.train(params, lgtrain, 10000, valid_sets=[lgval], early_stopping_rounds=250, verbose_eval=100)
    
#     pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_val_y = model.predict(valid_X, num_iteration=model.best_iteration)
    return model, pred_val_y

In [57]:
from datetime import *
List = ["totals_pageviews", "totals_hits", "visitNumber", "geoNetwork_country"]
for column_perm in List:
    for i in range(20):
        print(column_perm," ",i)
        train_permute = train.copy()
        train_permute[column_perm] = np.random.permutation(train_permute[column_perm])
        dev_df = train_permute[train_permute['date']<=20170531]
        val_df = train_permute[train_permute['date']>20170531]
        dev_y = np.log1p(dev_df["totals_transactionRevenue"].values)
        val_y = np.log1p(val_df["totals_transactionRevenue"].values)

        dev_X = dev_df[category_columns + numerical_colums] 
        val_X = val_df[category_columns + numerical_colums] 
        model, pred_val = permutation_test(dev_X, dev_y, val_X, val_y)

totals_pageviews   0


LightGBMError: Length of label is not same with #data

In [58]:
dev_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 765707 entries, 0 to 903652
Data columns (total 40 columns):
channelGrouping                                 765707 non-null int64
date                                            765707 non-null int64
fullVisitorId                                   765707 non-null object
sessionId                                       765707 non-null object
visitId                                         765707 non-null int64
visitNumber                                     765707 non-null int64
visitStartTime                                  765707 non-null int64
device_browser                                  765707 non-null int64
device_deviceCategory                           765707 non-null int64
device_isMobile                                 765707 non-null int64
device_operatingSystem                          765707 non-null int64
geoNetwork_city                                 765707 non-null int64
geoNetwork_continent                           

In [60]:
train["totals_bounces"].fillna(0, inplace=True)

In [63]:
train["totals_bounces"].fillna(0, inplace=True)
train["totals_newVisits"].fillna(0, inplace=True)
train["totals_pageviews"].fillna(0, inplace=True)
train["totals_bounces"].fillna(0, inplace=True)

In [64]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 40 columns):
channelGrouping                                 903653 non-null int64
date                                            903653 non-null int64
fullVisitorId                                   903653 non-null object
sessionId                                       903653 non-null object
visitId                                         903653 non-null int64
visitNumber                                     903653 non-null int64
visitStartTime                                  903653 non-null int64
device_browser                                  903653 non-null int64
device_deviceCategory                           903653 non-null int64
device_isMobile                                 903653 non-null int64
device_operatingSystem                          903653 non-null int64
geoNetwork_city                                 903653 non-null int64
geoNetwork_continent                           

In [65]:
train_x, valid_x, train_y, valid_y = train_test_split(train[features], train["totals_transactionRevenue"], 
                                     test_size=0.25, random_state=20)

def permutation_test(train_X, train_y, val_X, val_y):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "num_leaves" : 50,
        "min_child_samples" : 100,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1,
        "njobs" : 3
    }
    
    lgtrain = lgb.Dataset(train_x, label=train_y)
    lgval = lgb.Dataset(valid_x, label=valid_y)
    model = lgb.train(params, lgtrain, 10000, valid_sets=[lgval], early_stopping_rounds=250, verbose_eval=100)
    
#     pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_val_y = model.predict(valid_X, num_iteration=model.best_iteration)
    return model, pred_val_y

In [66]:
from datetime import *
List = ["totals_pageviews", "totals_hits", "visitNumber", "geoNetwork_country"]
for column_perm in List:
    for i in range(20):
        print(column_perm," ",i)
        train_permute = train.copy()
        train_permute[column_perm] = np.random.permutation(train_permute[column_perm])
        dev_df = train_permute[train_permute['date']<=20170531]
        val_df = train_permute[train_permute['date']>20170531]
        dev_y = np.log1p(dev_df["totals_transactionRevenue"].values)
        val_y = np.log1p(val_df["totals_transactionRevenue"].values)

        dev_X = dev_df[category_columns + numerical_colums] 
        val_X = val_df[category_columns + numerical_colums] 
        model, pred_val = permutation_test(dev_X, dev_y, val_X, val_y)

totals_pageviews   0


LightGBMError: Length of label is not same with #data

In [69]:
lgb_params = {"objective" : "regression", "metric" : "rmse",
              "num_leaves" : 50, "learning_rate" : 0.02, 
              "bagging_fraction" : 0.75, "feature_fraction" : 0.8, "bagging_frequency" : 9}
    
lgb_train = lgb.Dataset(train_x, label=train_y)
lgb_val = lgb.Dataset(valid_x, label=valid_y)
model = lgb.train(lgb_params, lgb_train, 700, valid_sets=[lgb_val], early_stopping_rounds=1000, verbose_eval=250)

Training until validation scores don't improve for 1000 rounds.
[250]	valid_0's rmse: 1.65654
[500]	valid_0's rmse: 1.65422
Did not meet early stopping. Best iteration is:
[584]	valid_0's rmse: 1.65401


In [68]:
    pred_val_y = model.predict(valid_x, num_iteration=model.best_iteration)
    pred_val_y

array([ 1.21677678e-02, -2.61379075e-04, -3.63743323e-03, ...,
        4.80294321e-02, -2.44224749e-03,  3.94914822e-01])

In [77]:
def permutation_test(train_X, train_y, val_X, val_y):
    lgb_params = {"objective" : "regression", "metric" : "rmse",
              "num_leaves" : 50, "learning_rate" : 0.02, 
              "bagging_fraction" : 0.75, "feature_fraction" : 0.8, "bagging_frequency" : 9, "bagging_seed" : 2018,
               "verbosity" : -1,"njobs" : 3}
    
    lgb_train = lgb.Dataset(train_X, label=train_y)
    lgb_val = lgb.Dataset(val_X, label=val_y)
    model = lgb.train(lgb_params, lgb_train, 700, valid_sets=[lgb_val], early_stopping_rounds=1000, verbose_eval=250)
    pred_val_y = model.predict(val_X, num_iteration=model.best_iteration)

    return model, pred_val_y

In [82]:
from datetime import *
List = ["totals_pageviews", "totals_hits", "visitNumber", "geoNetwork_country"]
for column_perm in List:
    for i in range(20):
        print(column_perm," ",i)
        train_permute = train.copy()
        train_permute[column_perm] = np.random.permutation(train_permute[column_perm])
        dev_df = train_permute[train_permute['date']<=20170531]
        val_df = train_permute[train_permute['date']>20170531]
        dev_y = np.log1p(dev_df["totals_transactionRevenue"].values)
        val_y = np.log1p(val_df["totals_transactionRevenue"].values)

        dev_X = dev_df[category_columns + numerical_colums] 
        val_X = val_df[category_columns + numerical_colums] 
        model, pred_val = permutation_test(dev_X, dev_y, val_X, val_y)

totals_pageviews   0


LightGBMError: Length of label is not same with #data

In [83]:
dev_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 765707 entries, 0 to 903652
Data columns (total 40 columns):
channelGrouping                                 765707 non-null int64
date                                            765707 non-null int64
fullVisitorId                                   765707 non-null object
sessionId                                       765707 non-null object
visitId                                         765707 non-null int64
visitNumber                                     765707 non-null int64
visitStartTime                                  765707 non-null int64
device_browser                                  765707 non-null int64
device_deviceCategory                           765707 non-null int64
device_isMobile                                 765707 non-null int64
device_operatingSystem                          765707 non-null int64
geoNetwork_city                                 765707 non-null int64
geoNetwork_continent                           

In [80]:
train["trafficSource_isTrueDirect"].fillna(0, inplace=True)

In [81]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 40 columns):
channelGrouping                                 903653 non-null int64
date                                            903653 non-null int64
fullVisitorId                                   903653 non-null object
sessionId                                       903653 non-null object
visitId                                         903653 non-null int64
visitNumber                                     903653 non-null int64
visitStartTime                                  903653 non-null int64
device_browser                                  903653 non-null int64
device_deviceCategory                           903653 non-null int64
device_isMobile                                 903653 non-null int64
device_operatingSystem                          903653 non-null int64
geoNetwork_city                                 903653 non-null int64
geoNetwork_continent                           

In [79]:
dev_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 765707 entries, 0 to 903652
Data columns (total 40 columns):
channelGrouping                                 765707 non-null int64
date                                            765707 non-null int64
fullVisitorId                                   765707 non-null object
sessionId                                       765707 non-null object
visitId                                         765707 non-null int64
visitNumber                                     765707 non-null int64
visitStartTime                                  765707 non-null int64
device_browser                                  765707 non-null int64
device_deviceCategory                           765707 non-null int64
device_isMobile                                 765707 non-null int64
device_operatingSystem                          765707 non-null int64
geoNetwork_city                                 765707 non-null int64
geoNetwork_continent                           

In [84]:
val_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137946 entries, 4822 to 901262
Data columns (total 40 columns):
channelGrouping                                 137946 non-null int64
date                                            137946 non-null int64
fullVisitorId                                   137946 non-null object
sessionId                                       137946 non-null object
visitId                                         137946 non-null int64
visitNumber                                     137946 non-null int64
visitStartTime                                  137946 non-null int64
device_browser                                  137946 non-null int64
device_deviceCategory                           137946 non-null int64
device_isMobile                                 137946 non-null int64
device_operatingSystem                          137946 non-null int64
geoNetwork_city                                 137946 non-null int64
geoNetwork_continent                        

In [85]:
dev_df = train_permute[train['date']<=20170531]
val_df = train_permute[train['date']>20170531]
dev_y = np.log1p(dev_df["totals_transactionRevenue"].values)
val_y = np.log1p(val_df["totals_transactionRevenue"].values)
dev_X = dev_df[category_columns + numerical_colums] 
val_X = val_df[category_columns + numerical_colums]

In [86]:
def permutation_test(train_X, train_y, val_X, val_y):
    lgb_params = {"objective" : "regression", "metric" : "rmse",
              "num_leaves" : 50, "learning_rate" : 0.02, 
              "bagging_fraction" : 0.75, "feature_fraction" : 0.8, "bagging_frequency" : 9, "bagging_seed" : 2018,
               "verbosity" : -1,"njobs" : 3}
    
    lgb_train = lgb.Dataset(train_X, label=train_y)
    lgb_val = lgb.Dataset(val_X, label=val_y)
    model = lgb.train(lgb_params, lgb_train, 700, valid_sets=[lgb_val], early_stopping_rounds=1000, verbose_eval=250)
    pred_val_y = model.predict(val_X, num_iteration=model.best_iteration)

    return model, pred_val_y

In [87]:
from datetime import *
List = ["totals_pageviews", "totals_hits", "visitNumber", "geoNetwork_country"]
for column_perm in List:
    for i in range(20):
        print(column_perm," ",i)
        train_permute = train.copy()
        train_permute[column_perm] = np.random.permutation(train_permute[column_perm])
        dev_df = train_permute[train_permute['date']<=20170531]
        val_df = train_permute[train_permute['date']>20170531]
        dev_y = np.log1p(dev_df["totals_transactionRevenue"].values)
        val_y = np.log1p(val_df["totals_transactionRevenue"].values)

        dev_X = dev_df[category_columns + numerical_colums] 
        val_X = val_df[category_columns + numerical_colums] 
        model, pred_val = permutation_test(dev_X, dev_y, val_X, val_y)

totals_pageviews   0
Training until validation scores don't improve for 1000 rounds.
[250]	valid_0's rmse: 0.293957
[500]	valid_0's rmse: 0.293376
Did not meet early stopping. Best iteration is:
[416]	valid_0's rmse: 0.293341
totals_pageviews   1
Training until validation scores don't improve for 1000 rounds.
[250]	valid_0's rmse: 0.293929
[500]	valid_0's rmse: 0.293402
Did not meet early stopping. Best iteration is:
[427]	valid_0's rmse: 0.293347
totals_pageviews   2
Training until validation scores don't improve for 1000 rounds.
[250]	valid_0's rmse: 0.294092
[500]	valid_0's rmse: 0.293641
Did not meet early stopping. Best iteration is:
[524]	valid_0's rmse: 0.29361
totals_pageviews   3
Training until validation scores don't improve for 1000 rounds.
[250]	valid_0's rmse: 0.293988
[500]	valid_0's rmse: 0.293477
Did not meet early stopping. Best iteration is:
[400]	valid_0's rmse: 0.293388
totals_pageviews   4
Training until validation scores don't improve for 1000 rounds.
[250]	valid_

Did not meet early stopping. Best iteration is:
[655]	valid_0's rmse: 0.286719
totals_hits   17
Training until validation scores don't improve for 1000 rounds.
[250]	valid_0's rmse: 0.287357
[500]	valid_0's rmse: 0.286783
Did not meet early stopping. Best iteration is:
[481]	valid_0's rmse: 0.286749
totals_hits   18
Training until validation scores don't improve for 1000 rounds.
[250]	valid_0's rmse: 0.287233
[500]	valid_0's rmse: 0.286552
Did not meet early stopping. Best iteration is:
[630]	valid_0's rmse: 0.2865
totals_hits   19
Training until validation scores don't improve for 1000 rounds.
[250]	valid_0's rmse: 0.287295
[500]	valid_0's rmse: 0.286548
Did not meet early stopping. Best iteration is:
[621]	valid_0's rmse: 0.286494
visitNumber   0
Training until validation scores don't improve for 1000 rounds.
[250]	valid_0's rmse: 0.283635
[500]	valid_0's rmse: 0.283071
Did not meet early stopping. Best iteration is:
[562]	valid_0's rmse: 0.28302
visitNumber   1
Training until valida

[250]	valid_0's rmse: 0.284245
[500]	valid_0's rmse: 0.283155
Did not meet early stopping. Best iteration is:
[590]	valid_0's rmse: 0.283052
geoNetwork_country   14
Training until validation scores don't improve for 1000 rounds.
[250]	valid_0's rmse: 0.284112
[500]	valid_0's rmse: 0.28323
Did not meet early stopping. Best iteration is:
[621]	valid_0's rmse: 0.283157
geoNetwork_country   15
Training until validation scores don't improve for 1000 rounds.
[250]	valid_0's rmse: 0.284216
[500]	valid_0's rmse: 0.28333
Did not meet early stopping. Best iteration is:
[639]	valid_0's rmse: 0.28329
geoNetwork_country   16
Training until validation scores don't improve for 1000 rounds.
[250]	valid_0's rmse: 0.284283
[500]	valid_0's rmse: 0.283315
Did not meet early stopping. Best iteration is:
[531]	valid_0's rmse: 0.283247
geoNetwork_country   17
Training until validation scores don't improve for 1000 rounds.
[250]	valid_0's rmse: 0.284279
[500]	valid_0's rmse: 0.283323
Did not meet early stoppi

In [88]:
cnt_srs = train.groupby('fullVisitorId')['totals_transactionRevenue'].agg(['size', 'count', 'mean'])

KeyError: 'Column not found: transactionRevenue'

In [92]:
train['Purchase'] =  train['totals_transactionRevenue'].apply(lambda x : [0,1][x>0])

In [94]:
cnt_srs = train.groupby('fullVisitorId')['Purchase'].agg(['size', 'sum'])

In [96]:
cnt_mean = train.groupby('fullVisitorId')['totals_transactionRevenue'].agg(['size', 'mean'])

In [97]:
cnt_srs['mean'] = cnt_mean['mean']

In [99]:
cnt_srs['prob'] = cnt_srs['sum']/cnt_srs['size']

In [102]:
sorted(cnt_srs)

['mean', 'prob', 'size', 'sum']

In [108]:
cnt_srs.sort_values(by=['prob'], ascending=False)

,size,sum,mean,prob
fullVisitorId,,,,
854434256389492978,1,1,17.488007,1.0
9799169384521023462,1,1,16.905188,1.0
8263279530140831487,1,1,17.280621,1.0
4339064982507273483,1,1,16.759423,1.0
2364782563241908647,1,1,16.873278,1.0
6463910848287283554,1,1,17.942161,1.0
6124362361380921120,1,1,18.155934,1.0
3943384499857599795,1,1,18.875190,1.0
646337888671074736,1,1,18.197037,1.0


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
spearman_correlation = train.corr(method='spearman')
pick_columns=spearman_correlation.nlargest(20, 'totals_transactionRevenue').index
correlationmap = np.corrcoef(train[pick_columns].values.T)
sns.set(font_scale=1.0)
heatmap = sns.heatmap(correlationmap, cbar=True, annot=True, square=True , fmt='.2f', 
                      yticklabels=train.values, xticklabels=train.values)
plt.show()